# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [3]:
from keras.datasets import cifar10
cf10 = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [6]:
print (cf10[0][0].shape)
print (cf10[1][0].shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
X_train = cf10[0][0]
Y_train = cf10[0][1]
X_test = cf10[1][0]
Y_test = cf10[1][1]

In [9]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [0]:
X1_train = []
Y1_train = []
X2_train = []
Y2_train = []
for i in np.arange(0,Y_train.shape[0]):
  if(Y_train[i] <= 4):
    Y1_train.append(Y_train[i])
    X1_train.append(X_train[i])
  elif(Y_train[i] >= 5):
    Y2_train.append(Y_train[i])
    X2_train.append(X_train[i])

In [0]:
X1_train = np.array(X1_train)
X2_train = np.array(X2_train)
Y1_train = np.array(Y1_train)
Y2_train = np.array(Y2_train)

In [13]:
print(X1_train.shape)
print(Y1_train.shape)
print(X2_train.shape)
print(Y2_train.shape)

(25000, 32, 32, 3)
(25000, 1)
(25000, 32, 32, 3)
(25000, 1)


In [0]:
X1_test = []
Y1_test = []
X2_test = []
Y2_test = []
for i in np.arange(0,Y_test.shape[0]):
  if(Y_test[i] <= 4):
    Y1_test.append(Y_test[i])
    X1_test.append(X_test[i])
  elif(Y_test[i] >= 5):
    Y2_test.append(Y_test[i])
    X2_test.append(X_test[i])

In [0]:
X1_test = np.array(X1_test)
X2_test = np.array(X2_test)
Y1_test = np.array(Y1_test)
Y2_test = np.array(Y2_test)

In [16]:
print(X1_test.shape)
print(Y1_test.shape)
print(X2_test.shape)
print(Y2_test.shape)

(5000, 32, 32, 3)
(5000, 1)
(5000, 32, 32, 3)
(5000, 1)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
Y11_train = tf.keras.utils.to_categorical(Y1_train,num_classes=10)
Y22_train = tf.keras.utils.to_categorical(Y2_train,num_classes=10)
Y11_test = tf.keras.utils.to_categorical(Y1_test,num_classes=10)
Y22_test = tf.keras.utils.to_categorical(Y2_test,num_classes=10)

In [18]:
print(Y11_train.shape)
print(Y22_train.shape)
print(Y11_test.shape)
print(Y22_test.shape)

(25000, 10)
(25000, 10)
(5000, 10)
(5000, 10)


In [0]:
X1_train = X1_train/255
X1_test = X1_test/255

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Convolution2D,MaxPool2D,Dropout,BatchNormalization,Activation,Flatten
from keras.optimizers import SGD
import keras.metrics

In [21]:
model = Sequential()
model.add(Convolution2D(filters=256,kernel_size=(3,3),input_shape=(32,32,3),))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Convolution2D(filters=128,kernel_size=(3,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Convolution2D(filters=64,kernel_size=(3,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10,activation='softmax'))
model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 256)       7168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       295040    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 6, 128)         512       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)         

In [23]:
model.fit(x=X1_train,y=Y11_train,batch_size=520,epochs=10,validation_data=(X1_test,Y11_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 13s 528us/step - loss: 2.0979 - acc: 0.2464 - val_loss: 2.0911 - val_acc: 0.3110
Epoch 2/10
25000/25000 [==============================] - 7s 290us/step - loss: 1.6913 - acc: 0.4470 - val_loss: 1.9492 - val_acc: 0.3958
Epoch 3/10
25000/25000 [==============================] - 7s 291us/step - loss: 1.4714 - acc: 0.5408 - val_loss: 1.5778 - val_acc: 0.4782
Epoch 4/10
25000/25000 [==============================] - 7s 288us/step - loss: 1.3276 - acc: 0.5862 - val_loss: 1.4041 - val_acc: 0.5384
Epoch 5/10
25000/25000 [==============================] - 7s 289us/step - loss: 1.2171 - acc: 0.6095 - val_loss: 1.2957 - val_acc: 0.5658
Epoch 6/10
25000/25000 [==============================] - 7s 288us/step - loss: 1.1357 - acc: 0.6307 - val_loss: 1.3645 - val_acc: 0.5178
Epoch 7/10
25000/25000 [==

In [0]:
model1 = Sequential()
model1.add(Convolution2D(filters=256,kernel_size=(3,3),input_shape=(32,32,3),))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(BatchNormalization())
model1.add(Convolution2D(filters=128,kernel_size=(3,3)))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(BatchNormalization())
model1.add(Convolution2D(filters=64,kernel_size=(3,3)))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(BatchNormalization())
model1.add(Flatten())
model1.add(Dense(1024,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(512,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(256,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(128,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(64,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(32,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(16,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(10,activation='softmax'))
model1.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 256)       7168      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 256)       0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 15, 15, 256)       1024      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 128)       295040    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 6, 6, 128)         512       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 64)         

In [26]:
model1.fit(x=X1_train,y=Y11_train,batch_size=520,epochs=20,validation_data=(X1_test,Y11_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 11s 421us/step - loss: 1.7644 - acc: 0.4604 - val_loss: 1.4990 - val_acc: 0.5664
Epoch 2/20
25000/25000 [==============================] - 8s 317us/step - loss: 1.1800 - acc: 0.6502 - val_loss: 1.7668 - val_acc: 0.5230
Epoch 3/20
25000/25000 [==============================] - 8s 318us/step - loss: 0.9373 - acc: 0.7065 - val_loss: 1.7102 - val_acc: 0.5584
Epoch 4/20
25000/25000 [==============================] - 8s 318us/step - loss: 0.7893 - acc: 0.7410 - val_loss: 1.5366 - val_acc: 0.5900
Epoch 5/20
25000/25000 [==============================] - 8s 320us/step - loss: 0.6539 - acc: 0.7801 - val_loss: 1.2313 - val_acc: 0.6130
Epoch 6/20
25000/25000 [==============================] - 8s 318us/step - loss: 0.5590 - acc: 0.8096 - val_loss: 1.1566 - val_acc: 0.6394
Epoch 7/20
25000/25000 [==============================] - 8s 320us/step - loss: 0.4763 - acc: 0.8360 - val_loss: 1.2623 - 

In [0]:
model2 = Sequential()
model2.add(Convolution2D(filters=256,kernel_size=(3,3),input_shape=(32,32,3),))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Convolution2D(filters=128,kernel_size=(3,3)))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Convolution2D(filters=64,kernel_size=(3,3)))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Flatten())
model2.add(Dense(1024,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(512,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(256,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(128,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(64,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(32,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(16,activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(10,activation='softmax'))
model2.compile(optimizer='adagrad',loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 30, 256)       7168      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 256)       0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 15, 15, 256)       1024      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 128)       295040    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 6, 6, 128)         512       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 64)         

In [29]:
model2.fit(x=X1_train,y=Y11_train,batch_size=200,epochs=10,validation_data=(X1_test,Y11_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 13s 514us/step - loss: 1.3267 - acc: 0.5613 - val_loss: 1.1761 - val_acc: 0.5868
Epoch 2/10
25000/25000 [==============================] - 10s 394us/step - loss: 0.8932 - acc: 0.6900 - val_loss: 0.8688 - val_acc: 0.6800
Epoch 3/10
25000/25000 [==============================] - 10s 392us/step - loss: 0.7230 - acc: 0.7425 - val_loss: 0.7695 - val_acc: 0.7230
Epoch 4/10
25000/25000 [==============================] - 10s 392us/step - loss: 0.5993 - acc: 0.7873 - val_loss: 0.7520 - val_acc: 0.7162
Epoch 5/10
25000/25000 [==============================] - 10s 394us/step - loss: 0.5024 - acc: 0.8207 - val_loss: 0.7227 - val_acc: 0.7382
Epoch 6/10
25000/25000 [==============================] - 10s 391us/step - loss: 0.4305 - acc: 0.8447 - val_loss: 0.6877 - val_acc: 0.7628
Epoch 7/10
25000/25000 [==============================] - 10s 392us/step - loss: 0.3478 - acc: 0.8786 - val_loss: 0.7

In [30]:
model2.fit(x=X1_train,y=Y11_train,batch_size=250,epochs=15,validation_data=(X1_test,Y11_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/15
25000/25000 [==============================] - 10s 385us/step - loss: 0.1184 - acc: 0.9634 - val_loss: 0.7597 - val_acc: 0.7836
Epoch 2/15
25000/25000 [==============================] - 9s 367us/step - loss: 0.0850 - acc: 0.9758 - val_loss: 0.7806 - val_acc: 0.7838
Epoch 3/15
25000/25000 [==============================] - 9s 366us/step - loss: 0.0666 - acc: 0.9826 - val_loss: 0.8361 - val_acc: 0.7798
Epoch 4/15
25000/25000 [==============================] - 9s 365us/step - loss: 0.0492 - acc: 0.9879 - val_loss: 0.8698 - val_acc: 0.7790
Epoch 5/15
25000/25000 [==============================] - 9s 364us/step - loss: 0.0413 - acc: 0.9898 - val_loss: 0.8730 - val_acc: 0.7864
Epoch 6/15
25000/25000 [==============================] - 9s 365us/step - loss: 0.0432 - acc: 0.9884 - val_loss: 0.9662 - val_acc: 0.7682
Epoch 7/15
25000/25000 [==============================] - 9s 365us/step - loss: 0.0396 - acc: 0.9898 - val_loss: 0.9047 - 

In [32]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
model2.save('/content/gdrive/My Drive/Model2-0-4.h5')

### Training and test data accuracy for cfar10 image data for 0-4 images using CNN



In [36]:
model2.evaluate(X1_test,Y11_test)

5000/5000 [==============================] - 2s 344us/step


[1.1536062109947205, 0.7732]

In [37]:
model2.evaluate(X1_train,Y11_train)

25000/25000 [==============================] - 8s 304us/step


[0.15179859247952698, 0.969]

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [38]:
tmodel = tf.keras.models.load_model('/content/gdrive/My Drive/Model2-0-4.h5')
tmodel.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 30, 256)       7168      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 256)       0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 15, 15, 256)       1024      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 128)       295040    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 128)         0         
_____________________________________________________

In [0]:
tmodel.compile(optimizer='adagrad',loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
for layer in tmodel.layers:
  if(layer.name.startswith('conv2d')):
    layer.trainable = False

In [41]:
for layer in tmodel.layers:
  print(layer.name)
  print(layer.trainable)

conv2d_7
False
max_pooling2d_7
True
batch_normalization_19
True
conv2d_8
False
max_pooling2d_8
True
batch_normalization_20
True
conv2d_9
False
max_pooling2d_9
True
batch_normalization_21
True
flatten_3
True
dense_15
True
batch_normalization_22
True
dense_16
True
batch_normalization_23
True
dense_17
True
batch_normalization_24
True
dense_18
True
batch_normalization_25
True
dense_19
True
batch_normalization_26
True
dense_20
True
batch_normalization_27
True
dense_21
True
batch_normalization_28
True
dense_22
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
X2_train = X2_train/255
X2_test = X2_test/255

In [43]:
tmodel.fit(x=X2_train,y=Y22_train,batch_size=520,epochs=10,validation_data=(X2_test,Y22_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 9s 354us/sample - loss: 4.6223 - acc: 0.0220 - val_loss: 3.7818 - val_acc: 0.0410
Epoch 2/10
25000/25000 [==============================] - 7s 286us/sample - loss: 2.7054 - acc: 0.1450 - val_loss: 2.7021 - val_acc: 0.1592
Epoch 3/10
25000/25000 [==============================] - 7s 287us/sample - loss: 2.1795 - acc: 0.2720 - val_loss: 2.4865 - val_acc: 0.2002
Epoch 4/10
25000/25000 [==============================] - 7s 288us/sample - loss: 1.9159 - acc: 0.3733 - val_loss: 2.1084 - val_acc: 0.3354
Epoch 5/10
25000/25000 [==============================] - 7s 288us/sample - loss: 1.7368 - acc: 0.4635 - val_loss: 1.8103 - val_acc: 0.5028
Epoch 6/10
25000/25000 [==============================] - 7s 289us/sample - loss: 1.5987 - acc: 0.5464 - val_loss: 1.8521 - val_acc: 0.5288
Epoch 7/10
25000/25000 [==============================] - 7s 289us/sample - loss: 1.4874 - acc: 0.6214 - val_lo

In [0]:
z = tmodel.output
z = tf.keras.layers.Dense(255,activation='relu')(z)
z = tf.keras.layers.Dense(127,activation='relu')(z)
z = tf.keras.layers.Dense(63,activation='relu')(z)
predictions = tf.keras.layers.Dense(10,activation='softmax')(z)
tmodel_final = tf.keras.models.Model(inputs=tmodel.input,outputs=predictions)
tmodel_final.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
tmodel_final.fit(x=X2_train,y=Y22_train,batch_size=520,epochs=10,validation_data=(X2_test,Y22_test))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 278us/sample - loss: 1.2858 - acc: 0.6864 - val_loss: 0.7201 - val_acc: 0.8056
Epoch 2/10
25000/25000 [==============================] - 5s 203us/sample - loss: 0.4881 - acc: 0.8456 - val_loss: 0.6398 - val_acc: 0.8118
Epoch 3/10
25000/25000 [==============================] - 5s 204us/sample - loss: 0.4079 - acc: 0.8670 - val_loss: 0.5184 - val_acc: 0.8430
Epoch 4/10
25000/25000 [==============================] - 5s 203us/sample - loss: 0.3537 - acc: 0.8840 - val_loss: 0.5151 - val_acc: 0.8374
Epoch 5/10
25000/25000 [==============================] - 5s 202us/sample - loss: 0.3216 - acc: 0.8952 - val_loss: 0.5201 - val_acc: 0.8406
Epoch 6/10
25000/25000 [==============================] - 5s 204us/sample - loss: 0.2754 - acc: 0.9109 - val_loss: 0.5177 - val_acc: 0.8480
Epoch 7/10
25000/25000 [==============================] - 5s 204us/sample - loss: 0.2426 - acc: 0.9182 - val_lo

###Training and Test data accuracy for cfar10 dataset images from 5-9 with model that was developed using Transfer learning

In [46]:
tmodel_final.evaluate(X2_test,Y22_test)

5000/5000 [==============================] - 2s 332us/sample - loss: 0.5793 - acc: 0.8394


[0.5792875322818756, 0.8394]

In [47]:
tmodel_final.evaluate(X2_train,Y22_train)

25000/25000 [==============================] - 8s 330us/sample - loss: 0.1586 - acc: 0.9483


[0.15861094340920448, 0.94828]

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [50]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [52]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [53]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [54]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf_train = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
dtmTrain = tfidf_train.fit_transform(twenty_train.data)

In [57]:
dtmTrain = dtmTrain.toarray()
dtmTrain

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
dtmTrain.shape

(2257, 266963)

In [59]:
dtmTest = tfidf_train.transform(twenty_test.data)
dtmTest = dtmTest.toarray()
dtmTest

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [60]:
dtmTest.shape

(1502, 266963)

In [0]:
Y_train = twenty_train.target
Y_test = twenty_test.target

In [62]:
print(Y_train.shape)
print(Y_test.shape)

(2257,)
(1502,)


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression

In [64]:
logR = LogisticRegression()
logR.fit(dtmTrain,Y_train)
print(logR.score(dtmTest,Y_test))
Y_pred = logR.predict(dtmTest)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8974700399467377


In [0]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [66]:
print(accuracy_score(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

0.8974700399467377
              precision    recall  f1-score   support

           0       0.96      0.77      0.86       319
           1       0.85      0.97      0.91       389
           2       0.94      0.86      0.90       396
           3       0.87      0.96      0.91       398

    accuracy                           0.90      1502
   macro avg       0.91      0.89      0.89      1502
weighted avg       0.90      0.90      0.90      1502

[[247  14  14  44]
 [  2 379   4   4]
 [  4  42 340  10]
 [  3  11   2 382]]
